# Explatory Data Analysis
## _Tidy up Data_

In [13]:
#Loading sql magic into python in order to use sql commands 
%load_ext sql
%config SqlMagic.autocommit=True

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [33]:
# importing pandas in order to use pd commands 
import pandas as pd

#importing ggplot
from lets_plot import *
LetsPlot.setup_html()


In [15]:
#Establishing the connection to plyalist.db. Located in data clean
%sql sqlite:///../data/clean/playlist.db --alias playlist

Connecting to 'playlist'

### __DataFrames__
#### United Kingdom 
- Creating a dataframe that has the categories, playlist id, and country it belongs to which 
- Country will be united kingdom for all

In [16]:
%%sql

df_uk <<

-- gets all the values for United Kingdom for analysis of categorie 
SELECT sub.categories, sub.id as playlist, sub.country as country
FROM 
    (SELECT t2.categories, t3.country, t1.id, t1.followers
    FROM playlist_name as t1
    INNER JOIN playlist_category as t2 
    ON t1.id = t2.playlist_id AND t1.country_id = t2.country_id
    INNER JOIN country as t3 
    ON t1.country_id = t3.id) sub
WHERE sub.country == 'United Kingdom'; 


Running query in 'playlist'

In [17]:
#Converting sql object to datafarme
df_uk = df_uk.DataFrame()

#### United States 
- Creating a dataframe that has the categories, playlist id, and country it belongs to which 
- Country will be United States for all

In [18]:
%%sql

df_us <<
--gets values for United States for category analysis
SELECT sub.categories, sub.id as playlist, sub.country as country 
FROM 
    (SELECT t2.categories, t3.country, t1.id, t1.followers
    FROM playlist_name as t1
    INNER JOIN playlist_category as t2 
    ON t1.id = t2.playlist_id AND t1.country_id = t2.country_id
    INNER JOIN country as t3 
    ON t1.country_id = t3.id) sub
WHERE sub.country == 'United States'; 

Running query in 'playlist'

In [19]:
# Converting sql object to a dataframe
df_us = df_us.DataFrame()



#### Mexico
- Creating a dataframe that has the categories, playlist id, and country it belongs to which 
- Country will be Mexico for all

In [20]:
%%sql

df_mexico <<
-- gets values for United States for category analysis and dumps into df_mexico
SELECT sub.categories, sub.id as playlist, sub.country as country 
FROM (SELECT t2.categories, t3.country, t1.id, t1.followers
    FROM playlist_name as t1
    INNER JOIN playlist_category as t2 
    ON t1.id = t2.playlist_id AND t1.country_id = t2.country_id
    INNER JOIN country as t3 
ON t1.country_id = t3.id) sub
WHERE sub.country == 'Mexico'; 

Running query in 'playlist'

In [21]:
#Converting sql object to dataframe
df_mexico = df_mexico.DataFrame()

#### Average Followers 
- Gets the average followers by country_id


In [22]:
%%sql

df_followers <<

-- Getting the average of the followers for each country id
SELECT SUM(followers)/COUNT(*) as Average, country_id
FROM(SELECT followers, id, country_id
    FROM playlist_name) as sub1
GROUP BY sub1.country_id;


Running query in 'playlist'

In [23]:
#Converting sql object to dataframe
df_followers = df_followers.DataFrame()

#### Category Count 
- Creates a dataframe that gets the count of the categories

In [24]:

%%sql 

df_category_count <<

--nmost popular categories in all playlists 
SELECT categories, COUNT(*) AS category_count
FROM playlist_category
GROUP BY categories
ORDEr BY category_count DESC;

Running query in 'playlist'

In [25]:
#Converts sql object to dataframe
df_category_count = df_category_count.DataFrame()

## *Analysis*

### Creating function called plot_data_country 
- Purpuse of function is to reduce amount of code written, produce plots of genres by their category for differnt countries. i.e. __mexico__, __united states__, and __united kingdom__.

In [36]:

#plotting the countries counts and categories
def plot_data_country (country_name, data): 
    

    df_copy = data.copy()


    # if a string contains pop then it will be convertede into pop 
    df_copy.loc[df_copy['categories'].str.contains('pop', case=False, na=False), 'categories'] = 'pop'

    # if a string contains rap then it will be converted to rap
    df_copy.loc[df_copy['categories'].str.contains('rap', case=False, na=False), 'categories'] = 'rap'

    # makes the value counts as a dataframe and resets the index
    country = pd.DataFrame(df_copy['categories'].value_counts()).reset_index()

    #renaming columns for analysis
    country = country.rename(columns={'index': 'category', 'categories':'count'})

    #creates a plot
    plot_df = country.copy()

    #limiting the counts for 6 times to ensure for clean plots 
    #many categories that appear once 
    plot_df = plot_df[plot_df['count'] >= 6]

    #Plots the value
    to_plot = (ggplot(data = plot_df)+ geom_bar(aes(x = 'category', y = 'count',fill = 'category'),stat = 'identity')+
    #sets the ledgend to none
    guides(fill = 'none')+ 
    labs(title = f"Featured Playlist of {country_name} by Genre", #F string that produces the title by the country_name
            y = 'Count',
            x = 'Category'
            
        
    )+ 
    #changes the theme, so that plot_title and axis titles fit the figure. 
    theme(plot_title=element_text(size=20, hjust=0.1, face = 'bold'), #hjust = 0.5 to adjust to the center of the plot 
          axis_title_x=element_text(size=20),
          axis_title_y=element_text(size=20)
          ))
    
    return to_plot

In [27]:

#making a copy of mexico dataframe 
df_mex = df_mexico.copy()

#calling the function
figure_mex = plot_data_country("Mexico", df_mex)

#saving the figure to an html file in figures 
ggsave(figure_mex, filename = '../../docs/figures/Mexico_Playlist.html')




'/Users/benpilnick/Documents/GitHub/ME204/me204-2024-project-benjaminpilnick/docs/figures/Mexico_Playlist.html'

In [28]:
figure_mex.show()

In [29]:
#defining a copy of the uk_dataframe
df_united_kingdom = df_uk.copy()

#Calling the function 
figure_uk = plot_data_country("United Kingdom", df_united_kingdom)

#saving the plot to an html file in figures
ggsave(figure_uk, filename = '../../docs/figures/United_Kingdom_Playlist.html')

'/Users/benpilnick/Documents/GitHub/ME204/me204-2024-project-benjaminpilnick/docs/figures/United_Kingdom_Playlist.html'

In [30]:
figure_uk.show()

In [31]:
#defining the united states dataframe 
df_united_states = df_us.copy()

#calling the function
figure_us = plot_data_country("United States", df_united_states)

#saving the figure to Figures
ggsave(figure_us, filename = '../../docs/figures/United_States_Playlist.html')

'/Users/benpilnick/Documents/GitHub/ME204/me204-2024-project-benjaminpilnick/docs/figures/United_States_Playlist.html'

In [32]:
figure_us.show()